### Step 1 - Install all Python Packages

In [ ]:
%pip install azure-storage-blob==12.22.0
%pip install azure-identity==1.17.1
%pip install azure-storage-blob==12.22.0
%pip install azure-ai-formrecognizer==3.3.3
%pip install python-dotenv==1.0.1

### Step 2 - Create Azure Client

Now we create all client needed for the training and testing of the model

In [4]:
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

load_dotenv()

blob_service_client = BlobServiceClient.from_connection_string(os.getenv('STORAGE_NAME_CNX_STRING'))    

creds = AzureKeyCredential(os.environ["FORM_RECOGNIZER_KEY"])

document_client = DocumentAnalysisClient(endpoint=os.environ["FORM_RECOGNIZER_ENDPOINT"],credential=creds)

### Step 3 - Create a container for training and upload all training assets

Create a container for training and upload all training assets

One the documents are uploaded, we can start the training process. 

For this go to https://documentintelligence.ai.azure.com/studio/

In [ ]:
import time

container_name="training"

container_client = blob_service_client.get_container_client(container_name)

if not container_client.exists():
    container_client.create_container()
    container_client = blob_service_client.get_container_client(container_name)
else:
    container_client.delete_container()
    time.sleep(120) # wait for the container to be deleted before creating it again
    blob_service_client.create_container(container_name)
    container_client = blob_service_client.get_container_client(container_name)

directory_path = "../documents/training"

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        print(filename)
        with open(file_path, "rb") as data:
            container_client.upload_blob(name=filename, data=data)

print("Now go to Document Intelligence Studio: https://documentintelligence.ai.azure.com/studio/")


### Step 4 - Test the custom model

**BE SURE TO TRAIN THE MODEL BEFORE RUNNING THIS CELL**

Before running this cell train the model using the studio https://documentintelligence.ai.azure.com/studio/ and follow the instruction inside the **README.md** file

In [ ]:
file_path = "../documents/test/test1.pdf"
model_name = "ModelV1"

with open(file_path, "rb") as data:
   poller = document_client.begin_analyze_document(model_name,data)

# Wait for the result, this is for demo purpose only, shouldn't be done in a production
# workload.
result = poller.result()

for idx, document in enumerate(result.documents):
    print(f"--------Analyzing document #{idx + 1}--------")
    print(f"Document has type {document.doc_type}")
    print(f"Document has document type confidence {document.confidence}")
    print(f"Document was analyzed with model with ID {result.model_id}")
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print(
            f"......found label '{name}' for field of type '{field.value_type}' with value '{field_value}' and with confidence {field.confidence}"
        )